# Easily export jupyter cells to python module
https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py

In [ ]:
! python /tf/src/scripts/notebook2script.py probability_evaluation.ipynb

In [1]:
from encoder import *
from data_loader import *
from model import *
from exp.nb_conditional import default_hparams
import math

In [2]:
cd /tf/src/data/gpt-2/

/tf/src/data/gpt-2


# Read in data

In [3]:
ds_path = "/tf/src/data/methods/DATA00M_[god-r]/test"
enc = get_encoder("117M", "models")
data_set = load_dataset(enc, ds_path, sample = 1000)
len(data_set)

  0%|          | 0/121596 [00:00<?, ?it/s]


1000

In [ ]:
val_context = tf.compat.v1.placeholder(tf.int32, [args.val_batch_size, None])
val_output = model(hparams=hparams, X=val_context)
val_loss        = tf.reduce_mean(
                                    input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
                                        labels=val_context[:, 1:], logits=val_output['logits'][:, :-1]))
def validation():
            print('Calculating validation loss...')
            losses = []
            for batch in tqdm.tqdm(val_batches):
                losses.append(sess.run(val_loss, feed_dict={val_context: batch}))

In [5]:
# API look
def cross_entropy(ds, chkpt_path, MAX_CHUNK = 1024, bs = 1):
    # Generate permutations (TODO)
    
    
    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        context = tf.compat.v1.placeholder(tf.int32, [bs, None])
        # Generate predictions of model
        output  = model(hparams=default_hparams(), X=context)
        # Calculate cross entropy using tf library
        loss1    = tf.reduce_mean(
            input_tensor = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = context[:, 1:], logits = output['logits'][:, :-1]
            )
        )
        loss2    = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels = context[:, 1:], logits = output['logits'][:, :-1]
        )
        
        # Load model
        saver = tf.compat.v1.train.Saver()
        ckpt = tf.train.latest_checkpoint(chkpt_path)
        saver.restore(sess, ckpt)
    
        # For each method:
        entropy = []
        for method in tqdm.tqdm(ds):
            enc_meth    = enc.encode(method)
            
            context_tokens = enc_meth[:MAX_CHUNK]
            val = sess.run(loss1, feed_dict={context: [context_tokens]})
            if not math.isnan(val):
                entropy.append(val)
#             for i in range(len(enc_meth) % MAX_CHUNK):
#                 context_tokens = enc_meth[MAX_CHUNK * i:MAX_CHUNK * (i + 1)]
#                 print(len(tok))
            for tok in enc_meth[MAX_CHUNK:]:
                context_tokens.append(tok)
                context_tokens.pop(0)
    
#                 # Need to recalculate this because this is not correct way (I think)
                val = sess.run(loss2, feed_dict={context: [context_tokens]})
                if not math.isnan(val[:, -1]):
                    entropy.append(val[:, -1])
        
        return entropy

In [8]:
# API look
chkpt_path = os.path.join("checkpoint", "m1")
entropy = cross_entropy(data_set[:2], chkpt_path)

100%|██████████| 2/2 [00:00<00:00,  2.88it/s]


In [7]:
entropy = np.array(entropy)
entropy.mean()

array([0.48468012], dtype=float32)

In [9]:
entropy = np.array(entropy)
entropy.mean()

1.4819677